# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [47]:
total_time_i = datetime.datetime.now()

### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



 ### Credits for the data
 
 [Moro et al., 2011] S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, Guimarães, Portugal, October, 2011. EUROSIS.

  Available at: [pdf] http://hdl.handle.net/1822/14838
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2011-esm-1.txt

### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [48]:
#The data are collected from 17 marketing campaigns

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [49]:
import pandas as pd

In [50]:
bank = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [51]:
bank

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

#### Business Objective

The ultimate goal of this data is to predict the clients who are most likely to predict whether a client will get a term deposit. This would allow the directed campaigns to be implemented more efficiently.

Presumably the cost of marketing campaigns is comparatively cheap and the bank would like to focus more on capturing potential clientele at the expense of marketing towards many people who are not likely customers. The paper says that the bank made use of human interlocuters so the cost will not be negligible but for the purposes of this experiment we will consider Recall to be more important than Precision.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [52]:
bank_reduced = bank[['age', "job", "marital", "education", "default", "housing", "loan", "y"]]#'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx']

In [53]:
categoricals = ["job", "marital", "education", "default", "housing", "loan"]#, "contact", "poutcome", "month", "day_of_week"]

s2 = pd.get_dummies(bank_reduced[categoricals])
bank_filtered = pd.concat([bank_reduced, s2], axis=1).drop(categoricals, axis=1)
bank_filtered

,age,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
0,56,no,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,57,no,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
2,37,no,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,40,no,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,56,no,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,yes,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41184,46,no,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
41185,56,no,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41186,44,yes,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [54]:
bank_filtered = bank_filtered.replace(["yes", "no", True, False], [1, 0, 1, 0]).dropna()
bank_filtered

,age,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,...,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes
0,56,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
1,57,0,0,0,0,0,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
2,37,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,40,0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
4,56,0,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41184,46,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
41185,56,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,1,1,0,0
41186,44,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [55]:
bank_filtered['y']

0        0
1        0
2        0
3        0
4        0
        ..
41183    1
41184    0
41185    0
41186    1
41187    0
Name: y, Length: 41188, dtype: object

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [56]:
X = bank_filtered.drop(['y'], axis=1)
y = bank_filtered['y']
y = y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

The baseline model should be to classify every client as the more-common class

In [57]:
defaultProb = y.sum()/y.count()
100*defaultProb

11.265417111780131

The baseline model should be one that says every customer is NOT going to open up an account. Presumably the upshot of that conclusion is of course that the bank should end the campaigns altogether.

The Precision of the pipeline would be 88.73% but the Recall would be 0% for identifying potential customers.

In [58]:
modelTimes = []
modelTrains = []
modelTests = []

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [59]:
time_i = datetime.datetime.now()
lgr = LogisticRegression(max_iter=10000).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i
lgr_train = lgr.score(X_train, y_train)
lgr_test = lgr.score(X_test, y_test)

modelTimes.append(str(time))
modelTrains.append(lgr_train)
modelTests.append(lgr_test)

### Problem 9: Score the Model

What is the accuracy of your model?

In [60]:
lgr_test

0.8897737205011168

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

#### KNN Model

In [61]:
time_i = datetime.datetime.now()
results = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier())]).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

knn_Train = results.score(X_train, y_train)
knn_Test = results.score(X_test, y_test)

modelTimes.append(str(time))
modelTrains.append(knn_Train)
modelTests.append(knn_Test)

#### Decision Tree

In [62]:
tree = DecisionTreeClassifier()
time_i = datetime.datetime.now()
results = tree.fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

DT_Train = results.score(X_train, y_train)
DT_Test = results.score(X_test, y_test)

modelTimes.append(str(time))
modelTrains.append(DT_Train)
modelTests.append(DT_Test)

#### SVM Models

In [63]:
time_i = datetime.datetime.now()
grid_svc = SVC().fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

svc_train = grid_svc.score(X_train, y_train)
svc_test = grid_svc.score(X_test, y_test)

modelTimes.append(str(time))
modelTrains.append(svc_train)
modelTests.append(svc_test)

In [64]:
res_dict = {'model': ['Logistic Regression', 'KNN', 'Decision Tree', 'SVC'],
           'train score': modelTrains,
           'test score': modelTests,
           'average fit time': modelTimes}
results_df = pd.DataFrame(res_dict).set_index('model')
results_df

,train score,test score,average fit time
model,,,
Logistic Regression,0.886537,0.889774,0:00:01.304981
KNN,0.889612,0.877731,0:00:00.089955
Decision Tree,0.917581,0.867049,0:00:00.271080
SVC,0.886537,0.889774,0:00:42.629779


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

#### Data Discussion

I am content with the current dataset and was careful in my approach to begin with. I had considered changin Month information to 0-11 representing the months but settled instead on one-hot encoding because I had concerns that the numerical features would fail to capture the "closeness" of November and January for example.

For a deeper exploration I will include all of the columns rather than truncating the dataset as above.

In [65]:
categoricals = ["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome", "month", "day_of_week"]

s2 = pd.get_dummies(bank[categoricals])
bank_filtered = pd.concat([bank, s2], axis=1).drop(categoricals, axis=1)
bank_filtered

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0


In [66]:
bank_filtered = bank_filtered.replace(["yes", "no", True, False], [1, 0, 1, 0]).dropna()
bank_filtered

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0


In [67]:
X = bank_filtered.drop(['y'], axis=1)
y = bank_filtered['y']
y = y.astype('int')
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [68]:
modelTimes2 = []
modelTrains2 = []
modelTests2 = []

In [69]:
time_i = datetime.datetime.now()
lgr = LogisticRegression(max_iter=10000).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i
lgr_train = lgr.score(X_train, y_train)
lgr_test = lgr.score(X_test, y_test)

modelTimes2.append(str(time))
modelTrains2.append(lgr_train)
modelTests2.append(lgr_test)

In [70]:
lgr_test

0.9128872487132175

#### KNN Model

In [71]:
time_i = datetime.datetime.now()
results = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier())]).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

knn_Train = results.score(X_train, y_train)
knn_Test = results.score(X_test, y_test)

modelTimes2.append(str(time))
modelTrains2.append(knn_Train)
modelTests2.append(knn_Test)

#### Decision Tree

In [72]:
tree = DecisionTreeClassifier()
time_i = datetime.datetime.now()
results = tree.fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

DT_Train = results.score(X_train, y_train)
DT_Test = results.score(X_test, y_test)

modelTimes2.append(str(time))
modelTrains2.append(DT_Train)
modelTests2.append(DT_Test)

#### SVM Models

In [73]:
time_i = datetime.datetime.now()
grid_svc = SVC().fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

svc_train = grid_svc.score(X_train, y_train)
svc_test = grid_svc.score(X_test, y_test)

modelTimes2.append(str(time))
modelTrains2.append(svc_train)
modelTests2.append(svc_test)

In [74]:
res_dict = {'model': ['Logistic Regression', 'KNN', 'Decision Tree', 'SVC'],
           'train score': modelTrains2,
           'test score': modelTests2,
           'average fit time': modelTimes2}
results_df = pd.DataFrame(res_dict).set_index('model')
results_df

,train score,test score,average fit time
model,,,
Logistic Regression,0.909812,0.912887,0:00:04.155922
KNN,0.919006,0.898805,0:00:00.101153
Decision Tree,1.000000,0.893464,0:00:00.660224
SVC,0.896798,0.901136,0:00:48.812487


I think it's interesting to note here that the substantially larger dataset affected train times differently for each model. The data are 64 columns as opposed to 35 when I do not truncate them, with the same number of rows. Nearly doubling the size of the data had the following impact on train times:

In [75]:
timeScale = []
for i in range(int(len(modelTimes))):
    #modelTimes[i+4].total_seconds()/modelTimes[i].total_seconds()
    time1_ = modelTimes[i].split(':')
    time2_ = modelTimes2[i].split(':')
    time1 = int(time1_[0])*3600. + int(time1_[1])*60. + float(time1_[2])
    time2 = int(time2_[0])*3600. + int(time2_[1])*60. + float(time2_[2])
    scale = time2/time1
    timeScale.append(scale)
scale_dict = {'model': ['Logistic Regression', 'KNN', 'Decision Tree', 'SVC'],
           'time scaling': timeScale}
scale_df = pd.DataFrame(scale_dict).set_index('model')
scale_df

,time scaling
model,
Logistic Regression,3.184661
KNN,1.124484
Decision Tree,2.435532
SVC,1.145033


This result seems to indicate that although Logistic Regression yields the model with the greatest Test Score, it may suffer as the dataset grows. SVC by contrast is very slow, but doesn't seem to suffer as much of a hit when datasets become very large.

In [76]:
modelTimes3 = []
modelTrains3 = []
modelTests3 = []

### Grid Search the Logistic Regression model

In [77]:
params = {"lgr__C":np.logspace(-3,3,7)}

In [78]:
lgr_pipe = Pipeline([('scale', StandardScaler()), ('lgr', LogisticRegression())]).fit(X_train, y_train)

In [79]:
time_i = datetime.datetime.now()
results = GridSearchCV(lgr_pipe, params).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i
lgr_train = lgr.score(X_train, y_train)
lgr_test = lgr.score(X_test, y_test)

modelTimes3.append(str(time))
modelTrains3.append(knn_Train)
modelTests3.append(knn_Test)

In [80]:
print(knn_Train, knn_Test)

0.9190055355928911 0.8988054773234923


### Grid Search the KNN model

In [81]:
params = {'knn__n_neighbors': [2,3,4,5,6]
         }

In [82]:
knn_pipe = Pipeline([('scale', StandardScaler()), ('knn', KNeighborsClassifier())]).fit(X_train, y_train)

In [83]:
time_i = datetime.datetime.now()
roc_grid = GridSearchCV(knn_pipe, param_grid=params)
results = roc_grid.fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

knn_Train = results.score(X_train, y_train)
knn_Test = results.score(X_test, y_test)

modelTimes3.append(str(time))
modelTrains3.append(knn_Train)
modelTests3.append(knn_Test)

### Grid Search the Decision Tree model

In [84]:
params = {'max_depth': [5,6],
         'min_samples_split': [2,3],
          'criterion': ['gini', 'entropy'],
          'min_samples_leaf': [2,3]
         }

In [85]:
tree = DecisionTreeClassifier()
time_i = datetime.datetime.now()
results = GridSearchCV(tree, params).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

DT_Train = results.score(X_train, y_train)
DT_Test = results.score(X_test, y_test)

modelTimes3.append(str(time))
modelTrains3.append(DT_Train)
modelTests3.append(DT_Test)

In [86]:
total_time_f = datetime.datetime.now()
total_time = total_time_f - total_time_i
print("So far this has taken:", total_time)

So far this has taken: 0:10:13.106248


### Grid Search the SVC model

In [87]:
params = {'kernel': ['rbf'], #['poly', 'linear'],
         'gamma': [1.0, 10.0]}
numParams = 2 * 2

In [88]:
time_i = datetime.datetime.now()
grid_svc = GridSearchCV(SVC(), param_grid=params).fit(X_train, y_train)
time_f = datetime.datetime.now()
time = time_f - time_i

svc_train = grid_svc.score(X_train, y_train)
svc_test = grid_svc.score(X_test, y_test)

modelTimes3.append(str(time))
modelTrains3.append(svc_train)
modelTests3.append(svc_test)

In [89]:
print(modelTrains3, modelTests3, modelTimes3)

[0.9190055355928911, 0.9168689909682431, 0.917516428733288, 1.0] [0.8988054773234923, 0.898902592988249, 0.915509371661649, 0.8926871904438186] ['0:00:16.746126', '0:02:18.356213', '0:00:21.294556', '1:46:35.974943']


In [90]:
res_dict = {'model': ['Logistic Regression', 'KNN', 'Decision Tree', 'SVC'],
           'train score': modelTrains3,
           'test score': modelTests3,
           'average fit time': modelTimes3}
results_df = pd.DataFrame(res_dict).set_index('model')
results_df

,train score,test score,average fit time
model,,,
Logistic Regression,0.919006,0.898805,0:00:16.746126
KNN,0.916869,0.898903,0:02:18.356213
Decision Tree,0.917516,0.915509,0:00:21.294556
SVC,1.000000,0.892687,1:46:35.974943


##### Questions